In [1]:
import torch
import numpy as np
import pandas as pd

In [16]:
data_df = pd.read_csv('train.csv')
data = data_df.to_numpy()
X, y = data[:, 1:], data[:, 0]

# PyTorch implementation of Mnist trainer

In [73]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split

In [37]:
device = torch.device("cpu")
BATCH_SIZE = 128
INPUT_SHAPE = (28, 28, 1)
NUM_CLASSES = 10
EPOCHS = 12

In [60]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

X_train = np.reshape(X_train, (X_train.shape[0],1, 28, 28))
X_test = np.reshape(X_test, (X_test.shape[0],1, 28, 28))


class MNISTDataset(Dataset):
    def __init__(self, X, y, device):
        """
        Args:
            X (matrix): X data
            y (matrix): y targets
            device (torch device): For converting our sample from a numpy 
                matrix to a pytorch tensor
        """
        self.device = device
        self.X = torch.tensor(X, dtype=torch.long, device=device)
        self.y = torch.tensor(y, dtype=torch.long, device=device)
    
    def __len__(self):
        return self.X.shape[0]
        
    def __getitem__(self, idx):
        target = self.y[idx]
        data_val = self.X[idx]
        return data_val.float(), target.float()
    
train_loader = torch.utils.data.DataLoader(
        MNISTDataset(X_train, y_train, device), 
        batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        MNISTDataset(X_test, y_test, device), 
        batch_size=BATCH_SIZE, shuffle=True)

In [61]:
class MnistNet(nn.Module):
    def __init__(self):
        super(MnistNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [70]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target.long())
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target.long(), reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [71]:
model = MnistNet().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.01)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, EPOCHS+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/28140 (0%)]	Loss: 17.481117
Train Epoch: 1 [1280/28140 (5%)]	Loss: 4.567411
Train Epoch: 1 [2560/28140 (9%)]	Loss: 2.749251
Train Epoch: 1 [3840/28140 (14%)]	Loss: 1.951471
Train Epoch: 1 [5120/28140 (18%)]	Loss: 1.449165
Train Epoch: 1 [6400/28140 (23%)]	Loss: 1.643783
Train Epoch: 1 [7680/28140 (27%)]	Loss: 1.417992
Train Epoch: 1 [8960/28140 (32%)]	Loss: 1.326936
Train Epoch: 1 [10240/28140 (36%)]	Loss: 1.541376
Train Epoch: 1 [11520/28140 (41%)]	Loss: 1.395081
Train Epoch: 1 [12800/28140 (45%)]	Loss: 1.078183
Train Epoch: 1 [14080/28140 (50%)]	Loss: 1.011986
Train Epoch: 1 [15360/28140 (55%)]	Loss: 0.958470
Train Epoch: 1 [16640/28140 (59%)]	Loss: 0.949952
Train Epoch: 1 [17920/28140 (64%)]	Loss: 0.987577
Train Epoch: 1 [19200/28140 (68%)]	Loss: 0.969934
Train Epoch: 1 [20480/28140 (73%)]	Loss: 0.713416
Train Epoch: 1 [21760/28140 (77%)]	Loss: 1.298281
Train Epoch: 1 [23040/28140 (82%)]	Loss: 0.965374
Train Epoch: 1 [24320/28140 (86%)]	Loss: 0.681320
Train Epoch: 

KeyboardInterrupt: 